
# Related keywords detection

**NOTE**: This notebook depends upon the the Retrotech dataset. If you have any issues, please rerun the [Setting up the Retrotech Dataset](../ch4/1.ch4-setting-up-the-retrotech-dataset.ipynb) notebook.

In [1]:
## Approcah (1): Utilize user and search query behavior data to find related keyword

import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("aips-ch6").getOrCreate()

### Step 1: Prepare the data using py-spark and data frames 
get candidate from wordnet if both query and candidate are included in wordnet. 

In [2]:
#use the real signals
signals_collection="signals"
signals_opts={"zkhost": "aips-zk", "collection": signals_collection}
df = spark.read.format("solr").options(**signals_opts).load()
df.createOrReplaceTempView("signals")

spark.sql(''' select query_id, target as query from signals where type='query' ''').show(3)

+-----------+--------------------+
|   query_id|               query|
+-----------+--------------------+
|u465908_0_1|transformers blueray|
| u59173_1_2|                SONY|
|u298681_0_1|          subwoofers|
+-----------+--------------------+
only showing top 3 rows



In [3]:
## MK: Sample data : each row is keyword, user (session), upc (product id)
#spark.sql('''
#select lower(query_s) as keyword, user_id_s, doc_id_s as upc from signals
#''').createOrReplaceTempView('keyword_click_product')

spark.sql("""select lower(searches.target) as keyword, searches.user as user, clicks.target as upc 
from signals as searches right join signals as clicks on searches.query_id = clicks.query_id 
where searches.type='query' and clicks.type = 'click'""").createOrReplaceTempView('keyword_click_product')


## Step2 : Create Cooccurrence & PMI2  Model 

In [4]:
### MK: Create co-occ model to compute the correlation between keywords based on the products,  
###Corr will be computed using pmi, pmi2
### (Keyword1, keywords, co-occ) co-occ is how many users uses these keywords with this product  
spark.sql('''
select k1.keyword as k1, k2.keyword as k2, sum(p1) n_users1,sum(p2) n_users1, 
sum(p1+p2) as users_cooc, count(1) n_products
from
(select keyword, upc, count(1) as p1 from keyword_click_product group by keyword, upc) as k1 
join
(select keyword, upc, count(1) as p2 from keyword_click_product group by keyword, upc) as k2
on k1.upc = k2.upc
where k1.keyword > k2.keyword 
group by k1.keyword, k2.keyword
''').createOrReplaceTempView('keyword_click_product_cooc')



In [5]:
## check co-occ table
spark.sql('''select count(1) as keyword_click_product_cooc from keyword_click_product_cooc''').show()

spark.sql('''select * from keyword_click_product_cooc order by n_products desc''').show(20)


+--------------------------+
|keyword_click_product_cooc|
+--------------------------+
|                   1579710|
+--------------------------+

+--------------+-------------+--------+--------+----------+----------+
|            k1|           k2|n_users1|n_users1|users_cooc|n_products|
+--------------+-------------+--------+--------+----------+----------+
|       laptops|       laptop|    3251|    3345|      6596|       187|
|       tablets|       tablet|    1510|    1629|      3139|       155|
|        tablet|         ipad|    1468|    7067|      8535|       146|
|       tablets|         ipad|    1359|    7048|      8407|       132|
|       cameras|       camera|     637|     688|      1325|       116|
|          ipad|        apple|    6706|    1129|      7835|       111|
|      iphone 4|       iphone|    1313|    1754|      3067|       108|
|    headphones|  head phones|    1829|     492|      2321|       106|
|     computers|     computer|     536|     392|       928|        98|
| 

In [6]:
## Mk: compute the pop of each keyword
spark.sql('''
select keyword, count(1) as n_users from keyword_click_product group by keyword
''').registerTempTable('keyword_click_product_oc')

spark.sql('''select count(1) as keyword_click_product_oc from keyword_click_product_oc''').show()


+------------------------+
|keyword_click_product_oc|
+------------------------+
|                   13744|
+------------------------+



In [7]:
##Mk: Based on co-occ and pop for each pair of the keywords , compute the PMI2
spark.sql('''
select k1.keyword as k1, k2.keyword as k2, k1_k2.users_cooc, k1.n_users as n_users1,k2.n_users as n_users2,
k1_k2.users_cooc/(k1.n_users*k2.n_users) as pmi2
from
keyword_click_product_cooc as k1_k2 
join
keyword_click_product_oc as k1 on k1_k2.k1 = k1.keyword
join
keyword_click_product_oc as k2 on k1_k2.k2 = k2.keyword
''').registerTempTable('related_keywords_pmi')


In [8]:
spark.sql('''select count(1) as related_keywords_pmi from related_keywords_pmi''').show()

+--------------------+
|related_keywords_pmi|
+--------------------+
|             1579710|
+--------------------+



In [ ]:
### MK: Get sample data from the table and filter the noisy signals based on Pop and PMI2. 
## TG to MK - What is this showing? I'm not sue if this data is super useful to display (doesn't look very clear)

data = spark.sql('''
select * from(
select k1, k2, pmi2, 
row_number() over (PARTITION BY k1 order by pmi2 desc ) rnum
from related_keywords_pmi 
where users_cooc > 10 and pmi2 > 0.01 
and k1 in ('lcd tv', 'ipad', 'laptop', 'iphone 4') ) x where rnum <= 20 ''').show(100)


In [10]:
#TG - average cooc with pmi2 for now to get a composite score:
#TODO @MK: can you replace with Composit rank score based upon position in list? from "jargon" paper?
spark.sql('''select *, (users_cooc + pmi2)/2 as score from related_keywords_pmi order by score desc''').show(1000)

+--------------------+--------------------+----------+--------+--------+--------------------+------------------+
|                  k1|                  k2|users_cooc|n_users1|n_users2|                pmi2|             score|
+--------------------+--------------------+----------+--------+--------+--------------------+------------------+
|                ipad|         hp touchpad|     11182|    7554|    4829|3.065386934784674...| 5591.000153269347|
|              ipad 2|                ipad|      9474|    2842|    7554|4.412983730371445...| 4737.000220649186|
|              tablet|                ipad|      8535|    1818|    7554|6.214878834984372E-4| 4267.500310743942|
|            touchpad|                ipad|      8455|    2785|    7554|4.018939161674483...| 4227.500200946958|
|             tablets|                ipad|      8407|    1627|    7554|6.840321494296586E-4| 4203.500342016075|
|               ipad2|                ipad|      7969|    1254|    7554|8.412581988101407E-4|398

### Approach 2:  Using NLP approach  


### Step 1: data cleaning
perform minimum stemming on queries and drop rare queries with count of 1

In [11]:
!pip install gensim
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet 
import re
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

     |████████████████████████████████| 24.1 MB 2.8 MB/s                                | 5.3 MB 7.3 MB/s eta 0:00:03
     |████████████████████████████████| 58 kB 5.7 MB/s             


[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [13]:
#TG to MK: Can you switch this to use the dataframe instead of the CSV, please? Data is already indexed into Solr.
csvFile = "../data/retrotech/products.csv"
import csv
reader = csv.reader(open(csvFile,'r'))
products = []
prod2 = []
for r in reader:
    if len(r)> 5:
        prod2.append(r)
        continue
    products.append(r)
    

In [14]:
# len(products), len(prod2)
products[0:2]

[['upc', 'name', 'manufacturer', 'shortDescription', 'longDescription'],
 ['096009010836', 'Fists of Bruce Lee - Dolby - DVD', '\\N', '\\N', '\\N']]

In [15]:
aggr_signals = aggr_signals[aggr_signals["count"] > 1]
aggr_signals.shape[0]

NameError: name 'aggr_signals' is not defined

### Step 2: find synonym from wordnet
get candidate from wordnet if both query and candidate are included in wordnet. 

In [ ]:
synonym = []
queries = []

for query in tqdm(aggr_signals["query"]):
    for synset in wordnet.synsets(query):
        for lemma in synset.lemmas():
            candidate = lemma.name().replace("_", " ")
            if candidate in list(aggr_signals["query"]) and candidate != query:
                queries.append(query)
                synonym.append(candidate)
                
wordnet_result = pd.DataFrame({"queries":queries,"synonym":synonym}).drop_duplicates()
wordnet_result.head()

### Step 3: find synonym from word2vec

In [ ]:
# combine queries and product descriptions to be fed into word2vec model
tokenizer = RegexpTokenizer(r'\w+')
tokenized_description = [tokenizer.tokenize(text.lower()) for text in product_description.longDescription]
tokenized_query = [tokenizer.tokenize(text) for text in aggr_signals["query"]]
tokenized_all = tokenized_query + tokenized_description

In [ ]:
# compute word2vec for each word
model = Word2Vec(tokenized_all, vector_size=150, window=8, min_count=1, workers=-1)
word2vec = dict(zip(model.wv.index_to_key, model.wv.vectors))

In [ ]:
# build word weights dictionary to weigh word vectors, so that rare words get more weights

tfidf = TfidfVectorizer(analyzer=lambda x: x)
tfidf.fit(tokenized_all)

max_idf = max(tfidf.idf_) #the default idf is the max of idf's for unseen words
weights = defaultdict(lambda: max_idf, [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

In [ ]:
query_vectors = []
tokenized_query_clean = [x for x in tokenized_query if x != []]
queries_cleaned = [" ".join(x) for x in tokenized_query_clean]

for tokens in tokenized_query_clean:
    tmp = []
    for token in tokens:
        if token in word2vec.keys():
            tmp.append(word2vec[token] * weights[token])
        else:
            tmp.append(np.zeros(150))
    query_vectors.append(np.mean(tmp, axis=0).tolist())

In [ ]:
knn = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute', n_jobs=-1)
knn.fit(np.stack(query_vectors))

In [ ]:
queries = []
synonym = []
cosine = []

for i in tqdm(range(len(queries_cleaned))):
    synonym_candidates = knn.kneighbors(np.expand_dims(query_vectors[i],axis=0), n_neighbors=6)
    query_index = synonym_candidates[1][0].tolist()[1:] #drop first candidate which is the same of the original query
    cosine_similarity = [1-x for x in synonym_candidates[0][0].tolist()[1:]]
    query = queries_cleaned[i]
    for j in range(0,5):
        queries.append(query)
        synonym.append(queries_cleaned[query_index[j]])
        cosine.append(cosine_similarity[j])

In [ ]:
synonym_candidates = pd.DataFrame({"queries":queries, "synonym":synonym, "cosine":cosine})
synonym_candidates.head(20)

In [ ]:
# apply cosine similarity threshold to further filter candidate synonyms
cosine_threshold = 0.7
word2vec_result = synonym_candidates[synonym_candidates.cosine >= cosine_threshold][["queries","synonym"]]
word2vec_result

### Step 4: combine wordnet and word2vec lists
the pair query A and synonym B is the same as the pair query B and synonym A, sort query and synonym to keep only 1 pair.

In [ ]:
combined_result = wordnet_result.append(word2vec_result)

def sort_pair (row): 
    return "_".join(sorted([row['queries'], row['synonym']])) 

combined_result["sorted_pair"] = combined_result.apply(lambda row: sort_pair(row), axis=1)
final_result = combined_result.groupby("sorted_pair").first().reset_index().drop(["sorted_pair"],axis=1)
len(final_result)

In [ ]:
final_result